## Import Libraries

In [15]:
import codecs
import spacy
import nltk
import numpy as np
# import torch
import matplotlib.pyplot as plt
import gensim, logging
import json
import h5py
# from wordcloud import WordCloud
import pandas as panda
from nltk.corpus import stopwords
import re
import tensorflow as tf
from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, BatchNormalization, Bidirectional
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.callbacks import Callback, ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import TimeDistributed
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.utils import class_weight as cw 

nlp = spacy.load("en")

In [75]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Read Training and Test Data

In [2]:
training_data = panda.read_csv("D:/Chirag B/AI/data/train.csv")

test_data = panda.read_csv("D:/Chirag B/AI/data/test.csv")

In [3]:
#checking for empty data values. If there are null values, a boolean array is returned. 
#We then sum all the boolean values(1 if null) to count the number of null values.
print(training_data.isnull().sum())
print(test_data.isnull().sum())


#replacing the null values with 'empty'
training_data = training_data.fillna(value='empty', axis=0)
test_data = test_data.fillna(value='empty', axis=0)

print ("########")

#verifying whether the dataframe still has null rows.
print(training_data.isnull().sum())
print(test_data.isnull().sum())

training_data

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64
test_id      0
question1    2
question2    4
dtype: int64
########
id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64
test_id      0
question1    0
question2    0
dtype: int64


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


## Clean Strings

In [4]:
def cleaner(question):

    
    #regex substitutions to clean question strings. This function is based on a discussion board in
    #Kaggle on 'The Importance of Cleaning Text'
    #https://www.kaggle.com/currie32/the-importance-of-cleaning-text/notebook
    

    
#     question = re.sub(r"[^A-Za-z0-9]", " ", question, flags=re.IGNORECASE)
    question = re.sub(r"'ve", " have ", question, flags=re.IGNORECASE)
    question = re.sub(r"what's", " what is ", question, flags=re.IGNORECASE)
    question = re.sub(r"whats", " what is ", question, flags=re.IGNORECASE)
    question = re.sub(r"can't", " cannot ", question, flags=re.IGNORECASE)
    question = re.sub(r"shan't", " shall not ", question, flags=re.IGNORECASE)
    question = re.sub(r"wouldn't", " would not ", question, flags=re.IGNORECASE)
    question = re.sub(r"hasn't", " has not ", question, flags=re.IGNORECASE)
    question = re.sub(r"couldn't", " could not ", question, flags=re.IGNORECASE)
    question = re.sub(r"couldnt", " could not ", question, flags=re.IGNORECASE)
    question = re.sub(r"didn't", " did not ", question, flags=re.IGNORECASE)
    question = re.sub(r"n't", " not", question, flags=re.IGNORECASE)
#     question = re.sub(r"I'm", "I am", question, flags=re.IGNORECASE)
    question = re.sub(r"\s{2,}", " ", question, flags=re.IGNORECASE)
#     question = re.sub(r",", " ", question, flags=re.IGNORECASE)
#     question = re.sub(r"\.", " ", question, flags=re.IGNORECASE)
    question = re.sub(r"\/", " / ", question, flags=re.IGNORECASE)
    question = re.sub(r"\+", " + ", question, flags=re.IGNORECASE)
    question = re.sub(r"\-", " - ", question, flags=re.IGNORECASE)
    question = re.sub(r"\*", " * ", question, flags=re.IGNORECASE)
    question = re.sub(r"\bu s\b", "American", question, flags=re.IGNORECASE)
    question = re.sub(r"\:", " : ", question, flags=re.IGNORECASE)
    question = re.sub(r"\?", "", question, flags=re.IGNORECASE)
    question = re.sub(r"e - mail", "email", question, flags=re.IGNORECASE)
    question = re.sub(r"(\d+)(k)\b", "\g<1>000", question, flags=re.IGNORECASE)
    question = re.sub(r"\0s", "0", question, flags=re.IGNORECASE)
    question = re.sub(r"\^", " ^ ", question, flags=re.IGNORECASE)
    question = re.sub(r"\=", " = ", question, flags=re.IGNORECASE)
    question = re.sub(r"\'ll", " will", question, flags=re.IGNORECASE)
    question = re.sub(r"\'d", " would ", question, flags=re.IGNORECASE)
    question = re.sub(r"\'re", " are", question, flags=re.IGNORECASE)
    question = re.sub(r" b g ", " bg ", question, flags=re.IGNORECASE)
    question = re.sub(r" e g ", " eg ", question, flags=re.IGNORECASE)
    question = re.sub(r" US ", " America ", question)
    question = re.sub(r" U.S.A ", " America ", question, flags=re.IGNORECASE)
    question = re.sub(r" U.S. ", " America ", question, flags=re.IGNORECASE)
    
    

    
    return question



In [5]:
#code to clean text
#making a call to "cleaner" for each row of both the train and test data
training_data['question1'] = training_data.apply((lambda x: cleaner(x['question1'])), axis=1)
training_data['question2'] = training_data.apply((lambda x: cleaner(x['question2'])), axis=1)

test_data['question1'] = test_data.apply((lambda x: cleaner(x['question1'])), axis=1)
test_data['question2'] = test_data.apply((lambda x: cleaner(x['question2'])), axis=1)


In [6]:
train = training_data
test = test_data
# train

print("TEST:", len(test))
print("TRAIN:", len(train))

TEST: 2345796
TRAIN: 404290


## Tokenization using Keras

In [7]:

#The tokenizer function is a Keras text preprocessing function. It is useful for creating sequences out of text sentences.
#The "tokenizr" object is made to "fit" on all strings/questions in the train and test dataset
tokenizr = Tokenizer(num_words = 200000)
tokenizer_train = [q1 for q1 in train["question1"]] + [q2 for q2 in train["question2"]] + [q3 for q3 in test["question1"]] + [q4 for q4 in test["question2"]]
tokenizr.fit_on_texts(tokenizer_train)
print("Done")

Done


## Computing GloVe embeddings for words

In [8]:
##GLOVE EMBEDDINGS

embed_dim = 300 #embedding dimensions
embed_file = codecs.open("glove.840B.300d/glove.840B.300d.txt", 'r', 'utf-8') #using GloVe common crawl embeddings

#index dictionary
#This dictionary stores a word as the key and its corresponding GloVe word vector as a value.
#This is useful for embedding text sequences.
vector_dict = {}  
for aline in embed_file:
    line = aline.split()
    
    try:
        word = line[0]
        word_vector = np.asarray(line[1:], dtype='float32')
    except:
        print("ISSUE --> ", line)
    vector_dict[word] = word_vector
embed_file.close()

print("Done --> ",len(vector_dict))


ISSUE -->  ['.', '.', '.', '-0.1573', '-0.29517', '0.30453', '-0.54773', '0.098293', '-0.1776', '0.21662', '0.19261', '-0.21101', '0.53788', '-0.047755', '0.40675', '0.023592', '-0.32814', '0.046858', '0.19367', '0.25565', '-0.021019', '-0.15957', '-0.1023', '0.20303', '-0.043333', '0.11618', '-0.18486', '0.0011948', '-0.052301', '0.34587', '0.052335', '0.16774', '-0.21384', '0.055947', '0.24934', '-0.12179', '0.16749', '0.28922', '-0.033739', '0.3015', '-0.13241', '0.092635', '0.37155', '-0.2884', '-0.0052731', '-0.001005', '-0.51153', '-0.28476', '-0.20139', '0.11837', '-0.0055891', '0.43604', '0.16796', '-0.2701', '0.063957', '-0.093253', '-0.22079', '0.36501', '0.06545', '0.23941', '-0.19292', '0.098293', '0.12172', '-0.1168', '-0.027436', '0.20507', '-0.39139', '-0.23111', '0.46239', '0.22888', '-0.028415', '-0.1798', '0.23817', '0.28093', '-0.47935', '0.23177', '-0.35587', '0.14246', '0.11861', '0.011018', '0.091986', '0.0054809', '-0.39955', '-0.40183', '-0.10629', '-0.30851', '

ISSUE -->  ['.', '.', '0.035974', '-0.024421', '0.71402', '-0.61127', '0.012771', '-0.11201', '0.16847', '-0.14069', '-0.053491', '-0.87539', '-0.13959', '0.29731', '0.072308', '-0.084514', '-0.1879', '0.12358', '0.37639', '-0.39238', '-0.01111', '-0.04924', '0.63649', '0.058814', '0.19076', '-0.20828', '-0.11036', '0.14934', '0.24667', '-0.39438', '0.22853', '-0.11201', '0.33539', '-0.32929', '-0.049727', '-0.090764', '0.29095', '0.27504', '0.22802', '-0.15616', '0.37302', '0.3752', '-0.3677', '0.1518', '-0.27551', '-0.63281', '-0.31298', '-0.22441', '-0.15435', '-0.64802', '0.28404', '0.12356', '0.0034255', '0.03094', '0.35345', '-0.46781', '0.59203', '-0.17966', '0.27702', '-0.46738', '0.19438', '0.21939', '-0.36743', '-0.084781', '0.03253', '-0.51323', '-0.55466', '0.49585', '0.066985', '0.47906', '-0.25118', '0.011123', '0.15605', '-1.0761', '0.60875', '-0.15764', '0.066122', '0.12779', '-0.089209', '0.4311', '0.045732', '-0.29364', '-0.19994', '-0.065952', '0.26236', '0.34039', '

ISSUE -->  ['contact', 'name@domain.com', '0.016426', '0.13728', '0.18781', '0.75784', '0.44012', '0.096794', '0.060987', '0.31293', '-0.15884', '-1.2367', '0.43769', '0.10465', '0.048858', '-0.23182', '0.71125', '0.022376', '0.63524', '-1.4974', '0.12243', '-0.07386', '-0.021514', '-0.37652', '0.17503', '-0.011225', '-0.12668', '-0.0090601', '0.38418', '0.11132', '0.15851', '-0.47498', '0.33619', '-0.48833', '0.23423', '0.13258', '0.29362', '0.13526', '-0.05115', '-0.0055236', '0.27734', '-0.23565', '0.19571', '-0.29095', '0.062419', '-0.47502', '-0.71402', '-0.36384', '0.53562', '0.40136', '0.30963', '0.16238', '-0.11662', '-0.16201', '0.30672', '0.21663', '0.086839', '-0.38895', '-0.19644', '-0.52311', '-0.33153', '0.27012', '-0.89654', '-0.15193', '0.12447', '-0.19112', '-0.494', '-0.011873', '-0.41412', '0.52585', '0.27316', '-0.047525', '-0.1178', '-0.3371', '0.61151', '-0.012169', '0.36935', '0.32679', '-0.098269', '0.038729', '0.003551', '-0.51871', '-0.48189', '-0.079238', '-0

ISSUE -->  ['by', 'name@domain.com', '0.6882', '-0.36436', '0.62079', '1.1482', '-0.055475', '-0.37936', '0.0064471', '-0.33046', '-0.43406', '-1.3468', '0.70312', '-0.41314', '-0.65868', '0.64324', '0.13018', '0.65846', '0.86269', '-0.93108', '0.3476', '0.73912', '-0.51405', '-0.15113', '0.27331', '0.51396', '-0.74688', '0.87989', '-0.11887', '0.3641', '0.37838', '0.36177', '-0.45182', '0.16173', '-0.36353', '-0.55643', '-1.1186', '0.70117', '-0.48075', '0.074095', '0.43022', '0.4625', '0.011133', '0.030287', '-0.73342', '-0.772', '0.31058', '0.022106', '-0.16845', '-0.70695', '-0.16243', '-0.15454', '-0.12034', '0.018702', '0.51626', '-0.17255', '0.37335', '-0.059377', '0.013126', '-0.30727', '0.1581', '0.74527', '-0.7927', '-0.34603', '-0.01438', '-1.055', '-0.95074', '-0.81794', '0.27925', '-0.35405', '-0.26783', '-0.30391', '0.16093', '-0.064806', '0.69283', '-1.1955', '0.18414', '-0.71183', '0.062622', '-0.62435', '-0.16458', '-0.74362', '-0.19251', '-0.1841', '0.99035', '-0.2055

ISSUE -->  ['at', 'name@domain.com', '0.44321', '-0.40005', '-0.20065', '1.1209', '0.34041', '0.086082', '-0.067128', '0.0022702', '-0.94649', '-1.4669', '0.61248', '0.34827', '-0.20983', '-0.61434', '0.41102', '0.57759', '0.69071', '-1.9301', '0.75265', '-0.13238', '0.22003', '0.28856', '0.35234', '0.45989', '-0.21944', '0.1931', '-0.11664', '0.14996', '0.70354', '-0.039238', '0.55298', '-0.53503', '-0.3221', '-0.28595', '-0.1246', '0.054544', '-0.45937', '0.1447', '0.8203', '-0.33182', '0.10864', '-0.56552', '0.39898', '-0.65012', '-0.20285', '0.11557', '0.35711', '-0.23958', '-0.30281', '0.51593', '0.71883', '-0.30403', '0.59458', '-0.3217', '-0.23967', '-0.2576', '-0.50224', '-0.36055', '-0.71763', '0.4981', '-0.69945', '-0.0072578', '0.37327', '-0.029839', '-0.42705', '0.93128', '-0.046928', '0.045162', '-0.44879', '0.16579', '-0.26272', '-0.35286', '0.17395', '-0.24436', '-0.1439', '-0.39857', '0.25342', '-0.44737', '0.37618', '-0.80252', '-0.87776', '-0.19282', '-0.48746', '0.06

## Create Sequences

In [2]:
#Creating text sequences for both the train and test questions.

train_sequence_q1 = train['question1'].tolist()
train_sequence_q2 = train['question2'].tolist()
test_sequence_q1 = test['question1'].tolist()
test_sequence_q2 = test['question2'].tolist()


#get test ID and validation labels.
train_labels = train['is_duplicate'].tolist()
test_id = test['test_id'].tolist()


#Using the text_to_sequences function as part of the Keras library(tokenizer class) for computing sequences.
# trained "tokeinzr" object on test and train dataset.
_train_sequence_q1 = tokenizr.texts_to_sequences(train_sequence_q1)
_train_sequence_q2 = tokenizr.texts_to_sequences(train_sequence_q2)
_test_sequence_q1 = tokenizr.texts_to_sequences(test_sequence_q1)
_test_sequence_q2 = tokenizr.texts_to_sequences(test_sequence_q2)


#padding sequences to be of same length 30
train_q1 = pad_sequences(_train_sequence_q1, maxlen = 30)
train_q2 = pad_sequences(_train_sequence_q2, maxlen = 30)
test_q1 = pad_sequences(_test_sequence_q1, maxlen = 30)
test_q2 = pad_sequences(_test_sequence_q2, maxlen = 30)

#creating a numpy array of labels for input to the LSTM neural network.
train_labels = np.array(train_labels)

#creating numpy array of test id to be used for the submission files in the end.
test_id = np.array(test_id)


# print (len(val_q1))
# X_train = 
# X_val =

## GloVe Matrix for Embedding Layer

In [41]:
#creating a GloVe embedding matrix for the embedding layer of the LSTM.
embed_mat = np.zeros((len(tokenizr.word_index)+1, embed_dim))
for key,value in tokenizr.word_index.items():
    embed_vector = vector_dict.get(key)
    if embed_vector is not None:
        embed_mat[value] = embed_vector


print("\nDone creating GloVe matrix\n")


Done creating GloVe matrix



## Implementation

In [4]:
#Parameters for the LSTM.
input_dim = len(tokenizr.word_index) + 1
output_dim = 300
input_len = 60
lstm_units = 250
drop = 0.4

In [42]:
#Read all features from the feature csv files.
feature_train = panda.read_csv("C:/dat/features_train.csv")
feature_test = panda.read_csv("C:/dat/features_test.csv")
feature_kcore_train = panda.read_csv("C:/dat/kcore_features_train.csv")
feature_kcore_test = panda.read_csv("C:/dat/kcore_features_test.csv")

In [43]:
#Get all training labels for input to LSTM
list_of_train_labels = feature_train['is_duplicate']

#Get labels for input to XGBoost
y_train = feature_train['is_duplicate'].values

#Drop 'id' and 'is_duplicate' fields from the input data
feature_train.drop(["id","is_duplicate"], axis=1, inplace=True)

#create a dataframe for the input.
result_1 = panda.concat([feature_train, feature_kcore_train], axis=1)
result_2 = panda.concat([feature_test, feature_kcore_test], axis=1)

#list of all training input data
list_of_all_training_features = result_1.values.tolist()
list_of_all_test_features = result_2.values.tolist()

#numpy array of input data. Used as input to the LSTM.
list_of_all_training_features = np.array(list_of_all_training_features)
list_of_all_test_features = np.array(list_of_all_test_features)

In [39]:
#numpy array of training labels
list_of_train_labels = np.array(list_of_train_labels)

# print(type(list_of_all_training_features))
test_id = test_data['test_id'].tolist()
test_id = np.array(test_id)

In [44]:
#Handling unbalanced classes using class weights from scikit-learn
y_labels = np.asarray(list_of_train_labels)

imbalance_count = np.count_nonzero(y_labels)

classes = np.unique(y_labels)

class_weights = cw.compute_class_weight('balanced', classes, y_labels)

print(len(y_labels))
print(imbalance_count)
print(classes)
print(class_weights)

# [ 0.79264156  1.3542874 ]
c_w = {0:1.3542874, 1:0.79264156 }


404290
149263
[0 1]
[ 0.79264156  1.3542874 ]


### Simple LSTM Experiment 1 - First ever neural network

In [12]:
#First neural network implementation
#Made use of tutorial on Machine Learning Mastery
#https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

#model definition
model = Sequential()
model.add(Embedding(input_dim, output_dim, weights=[embed_mat], input_length=input_len))
model.add(LSTM(250, dropout=0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

fit = model.fit(X_train, train_labels, epochs=200, batch_size=2000, shuffle=True)


scores = model.evaluate(X_val, val_labels, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))
print (type(scores))
print ("Loss : ", scores[0])
print ("Accuracy : ", scores[1]*100)

Epoch 1/150
323429/323429 [==============================] - ETA: 4:41 - loss: 0.6933 - acc: 0.512 - ETA: 2:51 - loss: 0.6832 - acc: 0.552 - ETA: 2:15 - loss: 0.6773 - acc: 0.572 - ETA: 1:57 - loss: 0.6728 - acc: 0.583 - ETA: 1:47 - loss: 0.6681 - acc: 0.594 - ETA: 1:39 - loss: 0.6665 - acc: 0.597 - ETA: 1:33 - loss: 0.6637 - acc: 0.601 - ETA: 1:29 - loss: 0.6602 - acc: 0.606 - ETA: 1:25 - loss: 0.6583 - acc: 0.608 - ETA: 1:23 - loss: 0.6564 - acc: 0.611 - ETA: 1:20 - loss: 0.6548 - acc: 0.613 - ETA: 1:18 - loss: 0.6524 - acc: 0.616 - ETA: 1:16 - loss: 0.6503 - acc: 0.617 - ETA: 1:15 - loss: 0.6490 - acc: 0.619 - ETA: 1:13 - loss: 0.6466 - acc: 0.622 - ETA: 1:12 - loss: 0.6434 - acc: 0.625 - ETA: 1:10 - loss: 0.6420 - acc: 0.627 - ETA: 1:09 - loss: 0.6398 - acc: 0.630 - ETA: 1:08 - loss: 0.6378 - acc: 0.632 - ETA: 1:07 - loss: 0.6363 - acc: 0.633 - ETA: 1:06 - loss: 0.6358 - acc: 0.634 - ETA: 1:06 - loss: 0.6345 - acc: 0.635 - ETA: 1:05 - loss: 0.6333 - acc: 0.636 - ETA: 1:04 - loss: 0

323429/323429 [==============================] - ETA: 1:07 - loss: 0.4878 - acc: 0.762 - ETA: 1:06 - loss: 0.4758 - acc: 0.765 - ETA: 1:05 - loss: 0.4701 - acc: 0.770 - ETA: 1:04 - loss: 0.4706 - acc: 0.768 - ETA: 1:04 - loss: 0.4682 - acc: 0.769 - ETA: 1:04 - loss: 0.4685 - acc: 0.771 - ETA: 1:03 - loss: 0.4685 - acc: 0.772 - ETA: 1:03 - loss: 0.4708 - acc: 0.771 - ETA: 1:03 - loss: 0.4695 - acc: 0.772 - ETA: 1:02 - loss: 0.4696 - acc: 0.771 - ETA: 1:02 - loss: 0.4715 - acc: 0.771 - ETA: 1:01 - loss: 0.4721 - acc: 0.771 - ETA: 1:01 - loss: 0.4717 - acc: 0.771 - ETA: 1:00 - loss: 0.4719 - acc: 0.771 - ETA: 1:00 - loss: 0.4708 - acc: 0.771 - ETA: 1:00 - loss: 0.4713 - acc: 0.770 - ETA: 59s - loss: 0.4712 - acc: 0.771 - ETA: 59s - loss: 0.4720 - acc: 0.77 - ETA: 59s - loss: 0.4724 - acc: 0.77 - ETA: 58s - loss: 0.4724 - acc: 0.77 - ETA: 58s - loss: 0.4729 - acc: 0.77 - ETA: 57s - loss: 0.4726 - acc: 0.77 - ETA: 57s - loss: 0.4732 - acc: 0.77 - ETA: 56s - loss: 0.4731 - acc: 0.77 - ETA: 5

323429/323429 [==============================] - ETA: 1:05 - loss: 0.4026 - acc: 0.810 - ETA: 1:04 - loss: 0.4052 - acc: 0.809 - ETA: 1:04 - loss: 0.4077 - acc: 0.808 - ETA: 1:04 - loss: 0.4103 - acc: 0.809 - ETA: 1:04 - loss: 0.4105 - acc: 0.810 - ETA: 1:03 - loss: 0.4090 - acc: 0.810 - ETA: 1:03 - loss: 0.4096 - acc: 0.808 - ETA: 1:03 - loss: 0.4095 - acc: 0.808 - ETA: 1:02 - loss: 0.4069 - acc: 0.809 - ETA: 1:02 - loss: 0.4072 - acc: 0.807 - ETA: 1:02 - loss: 0.4093 - acc: 0.806 - ETA: 1:01 - loss: 0.4093 - acc: 0.806 - ETA: 1:01 - loss: 0.4107 - acc: 0.806 - ETA: 1:00 - loss: 0.4094 - acc: 0.806 - ETA: 1:00 - loss: 0.4095 - acc: 0.806 - ETA: 1:00 - loss: 0.4096 - acc: 0.806 - ETA: 1:00 - loss: 0.4102 - acc: 0.805 - ETA: 59s - loss: 0.4111 - acc: 0.804 - ETA: 59s - loss: 0.4116 - acc: 0.80 - ETA: 58s - loss: 0.4117 - acc: 0.80 - ETA: 58s - loss: 0.4124 - acc: 0.80 - ETA: 57s - loss: 0.4127 - acc: 0.80 - ETA: 57s - loss: 0.4134 - acc: 0.80 - ETA: 57s - loss: 0.4136 - acc: 0.80 - ETA:

323429/323429 [==============================] - ETA: 1:07 - loss: 0.3605 - acc: 0.825 - ETA: 1:06 - loss: 0.3517 - acc: 0.831 - ETA: 1:05 - loss: 0.3544 - acc: 0.830 - ETA: 1:04 - loss: 0.3569 - acc: 0.831 - ETA: 1:04 - loss: 0.3568 - acc: 0.831 - ETA: 1:04 - loss: 0.3583 - acc: 0.830 - ETA: 1:03 - loss: 0.3592 - acc: 0.829 - ETA: 1:03 - loss: 0.3572 - acc: 0.829 - ETA: 1:03 - loss: 0.3564 - acc: 0.831 - ETA: 1:02 - loss: 0.3554 - acc: 0.833 - ETA: 1:02 - loss: 0.3563 - acc: 0.832 - ETA: 1:01 - loss: 0.3553 - acc: 0.832 - ETA: 1:01 - loss: 0.3557 - acc: 0.832 - ETA: 1:01 - loss: 0.3567 - acc: 0.832 - ETA: 1:00 - loss: 0.3569 - acc: 0.832 - ETA: 1:00 - loss: 0.3563 - acc: 0.832 - ETA: 59s - loss: 0.3565 - acc: 0.832 - ETA: 59s - loss: 0.3568 - acc: 0.83 - ETA: 59s - loss: 0.3558 - acc: 0.83 - ETA: 58s - loss: 0.3554 - acc: 0.83 - ETA: 58s - loss: 0.3552 - acc: 0.83 - ETA: 57s - loss: 0.3555 - acc: 0.83 - ETA: 57s - loss: 0.3554 - acc: 0.83 - ETA: 57s - loss: 0.3558 - acc: 0.83 - ETA: 5

323429/323429 [==============================] - ETA: 1:10 - loss: 0.3281 - acc: 0.842 - ETA: 1:07 - loss: 0.3206 - acc: 0.846 - ETA: 1:06 - loss: 0.3217 - acc: 0.850 - ETA: 1:05 - loss: 0.3268 - acc: 0.847 - ETA: 1:06 - loss: 0.3280 - acc: 0.846 - ETA: 1:05 - loss: 0.3266 - acc: 0.847 - ETA: 1:04 - loss: 0.3251 - acc: 0.847 - ETA: 1:03 - loss: 0.3238 - acc: 0.849 - ETA: 1:03 - loss: 0.3255 - acc: 0.847 - ETA: 1:03 - loss: 0.3271 - acc: 0.847 - ETA: 1:02 - loss: 0.3281 - acc: 0.847 - ETA: 1:02 - loss: 0.3269 - acc: 0.848 - ETA: 1:01 - loss: 0.3270 - acc: 0.848 - ETA: 1:01 - loss: 0.3266 - acc: 0.848 - ETA: 1:00 - loss: 0.3265 - acc: 0.848 - ETA: 1:00 - loss: 0.3267 - acc: 0.848 - ETA: 59s - loss: 0.3261 - acc: 0.848 - ETA: 59s - loss: 0.3250 - acc: 0.84 - ETA: 59s - loss: 0.3245 - acc: 0.84 - ETA: 58s - loss: 0.3243 - acc: 0.84 - ETA: 58s - loss: 0.3249 - acc: 0.84 - ETA: 57s - loss: 0.3241 - acc: 0.84 - ETA: 57s - loss: 0.3248 - acc: 0.84 - ETA: 57s - loss: 0.3245 - acc: 0.84 - ETA: 5

...

323429/323429 [==============================] - ETA: 1:08 - loss: 0.0255 - acc: 0.989 - ETA: 1:07 - loss: 0.0246 - acc: 0.991 - ETA: 1:07 - loss: 0.0258 - acc: 0.990 - ETA: 1:07 - loss: 0.0257 - acc: 0.990 - ETA: 1:07 - loss: 0.0268 - acc: 0.990 - ETA: 1:07 - loss: 0.0260 - acc: 0.990 - ETA: 1:07 - loss: 0.0268 - acc: 0.989 - ETA: 1:06 - loss: 0.0263 - acc: 0.990 - ETA: 1:06 - loss: 0.0259 - acc: 0.990 - ETA: 1:05 - loss: 0.0259 - acc: 0.990 - ETA: 1:05 - loss: 0.0267 - acc: 0.989 - ETA: 1:04 - loss: 0.0265 - acc: 0.989 - ETA: 1:04 - loss: 0.0262 - acc: 0.990 - ETA: 1:03 - loss: 0.0254 - acc: 0.990 - ETA: 1:03 - loss: 0.0258 - acc: 0.990 - ETA: 1:02 - loss: 0.0260 - acc: 0.990 - ETA: 1:02 - loss: 0.0258 - acc: 0.990 - ETA: 1:01 - loss: 0.0260 - acc: 0.990 - ETA: 1:01 - loss: 0.0260 - acc: 0.990 - ETA: 1:00 - loss: 0.0260 - acc: 0.990 - ETA: 1:00 - loss: 0.0265 - acc: 0.989 - ETA: 59s - loss: 0.0265 - acc: 0.990 - ETA: 59s - loss: 0.0263 - acc: 0.99 - ETA: 58s - loss: 0.0262 - acc: 0.9

323429/323429 [==============================] - ETA: 1:08 - loss: 0.0287 - acc: 0.990 - ETA: 1:07 - loss: 0.0303 - acc: 0.988 - ETA: 1:08 - loss: 0.0302 - acc: 0.988 - ETA: 1:07 - loss: 0.0284 - acc: 0.989 - ETA: 1:07 - loss: 0.0264 - acc: 0.989 - ETA: 1:06 - loss: 0.0260 - acc: 0.990 - ETA: 1:06 - loss: 0.0253 - acc: 0.990 - ETA: 1:05 - loss: 0.0257 - acc: 0.990 - ETA: 1:05 - loss: 0.0267 - acc: 0.989 - ETA: 1:04 - loss: 0.0260 - acc: 0.990 - ETA: 1:04 - loss: 0.0265 - acc: 0.990 - ETA: 1:03 - loss: 0.0260 - acc: 0.990 - ETA: 1:03 - loss: 0.0258 - acc: 0.990 - ETA: 1:03 - loss: 0.0251 - acc: 0.990 - ETA: 1:02 - loss: 0.0254 - acc: 0.990 - ETA: 1:02 - loss: 0.0256 - acc: 0.990 - ETA: 1:01 - loss: 0.0263 - acc: 0.990 - ETA: 1:01 - loss: 0.0263 - acc: 0.990 - ETA: 1:00 - loss: 0.0265 - acc: 0.990 - ETA: 1:00 - loss: 0.0266 - acc: 0.990 - ETA: 59s - loss: 0.0263 - acc: 0.990 - ETA: 59s - loss: 0.0262 - acc: 0.99 - ETA: 59s - loss: 0.0261 - acc: 0.99 - ETA: 58s - loss: 0.0266 - acc: 0.99 

323429/323429 [==============================] - ETA: 1:09 - loss: 0.0256 - acc: 0.989 - ETA: 1:08 - loss: 0.0279 - acc: 0.989 - ETA: 1:07 - loss: 0.0307 - acc: 0.987 - ETA: 1:06 - loss: 0.0282 - acc: 0.989 - ETA: 1:06 - loss: 0.0270 - acc: 0.989 - ETA: 1:05 - loss: 0.0253 - acc: 0.990 - ETA: 1:05 - loss: 0.0253 - acc: 0.990 - ETA: 1:05 - loss: 0.0264 - acc: 0.990 - ETA: 1:05 - loss: 0.0256 - acc: 0.990 - ETA: 1:04 - loss: 0.0250 - acc: 0.990 - ETA: 1:04 - loss: 0.0255 - acc: 0.990 - ETA: 1:03 - loss: 0.0263 - acc: 0.989 - ETA: 1:03 - loss: 0.0265 - acc: 0.989 - ETA: 1:03 - loss: 0.0269 - acc: 0.989 - ETA: 1:02 - loss: 0.0268 - acc: 0.989 - ETA: 1:02 - loss: 0.0269 - acc: 0.989 - ETA: 1:01 - loss: 0.0267 - acc: 0.989 - ETA: 1:01 - loss: 0.0268 - acc: 0.989 - ETA: 1:00 - loss: 0.0271 - acc: 0.989 - ETA: 1:00 - loss: 0.0274 - acc: 0.989 - ETA: 59s - loss: 0.0273 - acc: 0.989 - ETA: 59s - loss: 0.0268 - acc: 0.99 - ETA: 59s - loss: 0.0269 - acc: 0.98 - ETA: 58s - loss: 0.0272 - acc: 0.98 

323429/323429 [==============================] - ETA: 1:08 - loss: 0.0222 - acc: 0.990 - ETA: 1:07 - loss: 0.0299 - acc: 0.989 - ETA: 1:07 - loss: 0.0257 - acc: 0.990 - ETA: 1:06 - loss: 0.0249 - acc: 0.990 - ETA: 1:06 - loss: 0.0256 - acc: 0.990 - ETA: 1:05 - loss: 0.0247 - acc: 0.990 - ETA: 1:05 - loss: 0.0232 - acc: 0.991 - ETA: 1:04 - loss: 0.0250 - acc: 0.990 - ETA: 1:04 - loss: 0.0255 - acc: 0.990 - ETA: 1:04 - loss: 0.0258 - acc: 0.990 - ETA: 1:03 - loss: 0.0252 - acc: 0.990 - ETA: 1:03 - loss: 0.0250 - acc: 0.990 - ETA: 1:03 - loss: 0.0244 - acc: 0.990 - ETA: 1:02 - loss: 0.0250 - acc: 0.990 - ETA: 1:02 - loss: 0.0254 - acc: 0.990 - ETA: 1:01 - loss: 0.0257 - acc: 0.990 - ETA: 1:01 - loss: 0.0258 - acc: 0.990 - ETA: 1:01 - loss: 0.0256 - acc: 0.990 - ETA: 1:00 - loss: 0.0256 - acc: 0.990 - ETA: 1:00 - loss: 0.0253 - acc: 0.990 - ETA: 59s - loss: 0.0254 - acc: 0.990 - ETA: 59s - loss: 0.0251 - acc: 0.99 - ETA: 59s - loss: 0.0250 - acc: 0.99 - ETA: 58s - loss: 0.0249 - acc: 0.99 

323429/323429 [==============================] - ETA: 1:08 - loss: 0.0263 - acc: 0.988 - ETA: 1:07 - loss: 0.0251 - acc: 0.989 - ETA: 1:07 - loss: 0.0265 - acc: 0.989 - ETA: 1:06 - loss: 0.0237 - acc: 0.991 - ETA: 1:06 - loss: 0.0246 - acc: 0.990 - ETA: 1:06 - loss: 0.0236 - acc: 0.990 - ETA: 1:05 - loss: 0.0243 - acc: 0.990 - ETA: 1:05 - loss: 0.0245 - acc: 0.990 - ETA: 1:05 - loss: 0.0247 - acc: 0.990 - ETA: 1:04 - loss: 0.0245 - acc: 0.990 - ETA: 1:04 - loss: 0.0247 - acc: 0.990 - ETA: 1:03 - loss: 0.0246 - acc: 0.990 - ETA: 1:03 - loss: 0.0245 - acc: 0.990 - ETA: 1:02 - loss: 0.0247 - acc: 0.990 - ETA: 1:02 - loss: 0.0246 - acc: 0.990 - ETA: 1:01 - loss: 0.0247 - acc: 0.990 - ETA: 1:01 - loss: 0.0247 - acc: 0.990 - ETA: 1:01 - loss: 0.0246 - acc: 0.990 - ETA: 1:00 - loss: 0.0250 - acc: 0.990 - ETA: 1:00 - loss: 0.0251 - acc: 0.990 - ETA: 1:00 - loss: 0.0255 - acc: 0.990 - ETA: 59s - loss: 0.0254 - acc: 0.990 - ETA: 59s - loss: 0.0255 - acc: 0.99 - ETA: 58s - loss: 0.0255 - acc: 0.9

### Siamese LSTM Experiment 1 (Just GloVe vectors)

In [14]:
#For this Siamese experiment we use only 300 dimension GloVe embedding vectors for sentences. No other features.

#siamese parameters
in_dim = len(tokenizr.word_index) + 1
out_dim = 300
_len = 30
_units = 250
_drop = 0.1

In [9]:
#embedding layer which uses previously copmuted GloVe embedding matrix
embed_layer = Embedding(in_dim, out_dim, weights=[embed_mat], input_length=_len)

#LSTM layers. Outputs tensors.
lstm = LSTM(_units, dropout=_drop, recurrent_dropout=_drop)

#input sequences
question_1 = Input(shape=(30,), dtype="int32")
question_2 = Input(shape=(30,), dtype="int32")



#embedding the two input sequences
emb_question_1 = embed_layer(question_1)
emb_question_2 = embed_layer(question_2)


#create representations
_question_1 = lstm(emb_question_1)
_question_2 = lstm(emb_question_2)


#merge the two representations
_merge = concatenate([_question_1, _question_2])
_merge = Dense(150, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge) #dropout layer to prevent overfitting
_merge = BatchNormalization()(_merge)
_merge = Dense(150, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge) #dropout layer to prevent overfitting
_merge = BatchNormalization()(_merge)


is_duplicate = Dense(1, activation='sigmoid')(_merge)



model = Model(inputs=[question_1, question_2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# stop = EarlyStopping(monitor='val_loss', patience=2)
# checkpoint = ModelCheckpoint('model1.h5', monitor='val_acc', save_best_only=True, save_weights_only=False)

fit = model.fit([train_q1, train_q2], train_labels, epochs=50, batch_size=2000, shuffle=True)


score = model.evaluate([val_q1, val_q2], val_labels, verbose=0)

print("\n\n")
print("LOSS : ", score[0])
print("Accuracy : ", score[1]*100)

Epoch 1/25
323429/323429 [==============================] - ETA: 8:29 - loss: 0.8864 - acc: 0.510 - ETA: 4:38 - loss: 0.8522 - acc: 0.519 - ETA: 3:21 - loss: 0.8283 - acc: 0.533 - ETA: 2:42 - loss: 0.8161 - acc: 0.543 - ETA: 2:19 - loss: 0.8033 - acc: 0.551 - ETA: 2:03 - loss: 0.7861 - acc: 0.561 - ETA: 1:52 - loss: 0.7714 - acc: 0.569 - ETA: 1:43 - loss: 0.7590 - acc: 0.575 - ETA: 1:36 - loss: 0.7505 - acc: 0.581 - ETA: 1:31 - loss: 0.7413 - acc: 0.585 - ETA: 1:26 - loss: 0.7322 - acc: 0.590 - ETA: 1:23 - loss: 0.7256 - acc: 0.593 - ETA: 1:19 - loss: 0.7185 - acc: 0.597 - ETA: 1:16 - loss: 0.7132 - acc: 0.600 - ETA: 1:14 - loss: 0.7074 - acc: 0.604 - ETA: 1:12 - loss: 0.7030 - acc: 0.607 - ETA: 1:10 - loss: 0.6984 - acc: 0.610 - ETA: 1:08 - loss: 0.6944 - acc: 0.613 - ETA: 1:06 - loss: 0.6893 - acc: 0.616 - ETA: 1:05 - loss: 0.6851 - acc: 0.620 - ETA: 1:03 - loss: 0.6814 - acc: 0.622 - ETA: 1:02 - loss: 0.6784 - acc: 0.624 - ETA: 1:01 - loss: 0.6753 - acc: 0.627 - ETA: 59s - loss: 0.6

323429/323429 [==============================] - ETA: 53s - loss: 0.4072 - acc: 0.80 - ETA: 51s - loss: 0.4042 - acc: 0.80 - ETA: 51s - loss: 0.4048 - acc: 0.80 - ETA: 51s - loss: 0.4018 - acc: 0.80 - ETA: 51s - loss: 0.4039 - acc: 0.80 - ETA: 50s - loss: 0.4012 - acc: 0.80 - ETA: 50s - loss: 0.3980 - acc: 0.80 - ETA: 50s - loss: 0.3982 - acc: 0.80 - ETA: 49s - loss: 0.3964 - acc: 0.80 - ETA: 49s - loss: 0.3976 - acc: 0.80 - ETA: 49s - loss: 0.3972 - acc: 0.80 - ETA: 48s - loss: 0.3959 - acc: 0.80 - ETA: 48s - loss: 0.3956 - acc: 0.81 - ETA: 47s - loss: 0.3968 - acc: 0.80 - ETA: 47s - loss: 0.3963 - acc: 0.81 - ETA: 47s - loss: 0.3966 - acc: 0.80 - ETA: 47s - loss: 0.3966 - acc: 0.80 - ETA: 46s - loss: 0.3978 - acc: 0.80 - ETA: 46s - loss: 0.3979 - acc: 0.80 - ETA: 46s - loss: 0.3975 - acc: 0.80 - ETA: 45s - loss: 0.3972 - acc: 0.80 - ETA: 45s - loss: 0.3980 - acc: 0.80 - ETA: 45s - loss: 0.3979 - acc: 0.80 - ETA: 44s - loss: 0.3979 - acc: 0.80 - ETA: 44s - loss: 0.3978 - acc: 0.80 - E

323429/323429 [==============================] - ETA: 53s - loss: 0.3108 - acc: 0.85 - ETA: 53s - loss: 0.3204 - acc: 0.85 - ETA: 52s - loss: 0.3200 - acc: 0.85 - ETA: 51s - loss: 0.3215 - acc: 0.85 - ETA: 51s - loss: 0.3207 - acc: 0.85 - ETA: 50s - loss: 0.3175 - acc: 0.85 - ETA: 50s - loss: 0.3175 - acc: 0.85 - ETA: 50s - loss: 0.3154 - acc: 0.85 - ETA: 50s - loss: 0.3143 - acc: 0.85 - ETA: 50s - loss: 0.3133 - acc: 0.85 - ETA: 49s - loss: 0.3151 - acc: 0.85 - ETA: 49s - loss: 0.3147 - acc: 0.85 - ETA: 48s - loss: 0.3147 - acc: 0.85 - ETA: 48s - loss: 0.3139 - acc: 0.85 - ETA: 48s - loss: 0.3125 - acc: 0.86 - ETA: 47s - loss: 0.3136 - acc: 0.86 - ETA: 47s - loss: 0.3125 - acc: 0.86 - ETA: 46s - loss: 0.3140 - acc: 0.86 - ETA: 46s - loss: 0.3135 - acc: 0.86 - ETA: 46s - loss: 0.3136 - acc: 0.85 - ETA: 45s - loss: 0.3129 - acc: 0.86 - ETA: 45s - loss: 0.3130 - acc: 0.86 - ETA: 45s - loss: 0.3132 - acc: 0.85 - ETA: 44s - loss: 0.3128 - acc: 0.86 - ETA: 44s - loss: 0.3124 - acc: 0.86 - E

323429/323429 [==============================] - ETA: 54s - loss: 0.2486 - acc: 0.90 - ETA: 52s - loss: 0.2493 - acc: 0.89 - ETA: 52s - loss: 0.2513 - acc: 0.89 - ETA: 51s - loss: 0.2510 - acc: 0.89 - ETA: 51s - loss: 0.2523 - acc: 0.89 - ETA: 51s - loss: 0.2516 - acc: 0.89 - ETA: 50s - loss: 0.2521 - acc: 0.89 - ETA: 50s - loss: 0.2532 - acc: 0.89 - ETA: 49s - loss: 0.2521 - acc: 0.89 - ETA: 49s - loss: 0.2537 - acc: 0.89 - ETA: 49s - loss: 0.2538 - acc: 0.89 - ETA: 48s - loss: 0.2544 - acc: 0.88 - ETA: 48s - loss: 0.2541 - acc: 0.88 - ETA: 48s - loss: 0.2537 - acc: 0.88 - ETA: 47s - loss: 0.2549 - acc: 0.88 - ETA: 47s - loss: 0.2538 - acc: 0.88 - ETA: 47s - loss: 0.2534 - acc: 0.88 - ETA: 46s - loss: 0.2534 - acc: 0.88 - ETA: 46s - loss: 0.2531 - acc: 0.88 - ETA: 46s - loss: 0.2533 - acc: 0.88 - ETA: 45s - loss: 0.2537 - acc: 0.88 - ETA: 45s - loss: 0.2525 - acc: 0.88 - ETA: 45s - loss: 0.2522 - acc: 0.88 - ETA: 44s - loss: 0.2520 - acc: 0.88 - ETA: 44s - loss: 0.2514 - acc: 0.88 - E

323429/323429 [==============================] - ETA: 52s - loss: 0.2033 - acc: 0.91 - ETA: 52s - loss: 0.2114 - acc: 0.91 - ETA: 51s - loss: 0.2144 - acc: 0.90 - ETA: 51s - loss: 0.2130 - acc: 0.90 - ETA: 51s - loss: 0.2150 - acc: 0.90 - ETA: 50s - loss: 0.2125 - acc: 0.90 - ETA: 50s - loss: 0.2095 - acc: 0.90 - ETA: 50s - loss: 0.2093 - acc: 0.90 - ETA: 49s - loss: 0.2095 - acc: 0.90 - ETA: 49s - loss: 0.2095 - acc: 0.90 - ETA: 49s - loss: 0.2096 - acc: 0.90 - ETA: 48s - loss: 0.2086 - acc: 0.90 - ETA: 48s - loss: 0.2094 - acc: 0.90 - ETA: 48s - loss: 0.2088 - acc: 0.90 - ETA: 47s - loss: 0.2077 - acc: 0.91 - ETA: 47s - loss: 0.2071 - acc: 0.91 - ETA: 47s - loss: 0.2071 - acc: 0.91 - ETA: 46s - loss: 0.2069 - acc: 0.91 - ETA: 46s - loss: 0.2058 - acc: 0.91 - ETA: 46s - loss: 0.2058 - acc: 0.91 - ETA: 45s - loss: 0.2055 - acc: 0.91 - ETA: 45s - loss: 0.2053 - acc: 0.91 - ETA: 45s - loss: 0.2053 - acc: 0.91 - ETA: 44s - loss: 0.2056 - acc: 0.91 - ETA: 44s - loss: 0.2060 - acc: 0.91 - E

...

323429/323429 [==============================] - ETA: 52s - loss: 0.1026 - acc: 0.95 - ETA: 52s - loss: 0.1118 - acc: 0.95 - ETA: 51s - loss: 0.1079 - acc: 0.95 - ETA: 50s - loss: 0.1077 - acc: 0.95 - ETA: 50s - loss: 0.1064 - acc: 0.95 - ETA: 50s - loss: 0.1055 - acc: 0.95 - ETA: 49s - loss: 0.1064 - acc: 0.95 - ETA: 49s - loss: 0.1051 - acc: 0.95 - ETA: 49s - loss: 0.1058 - acc: 0.95 - ETA: 48s - loss: 0.1059 - acc: 0.95 - ETA: 48s - loss: 0.1046 - acc: 0.95 - ETA: 48s - loss: 0.1047 - acc: 0.95 - ETA: 47s - loss: 0.1043 - acc: 0.95 - ETA: 47s - loss: 0.1041 - acc: 0.95 - ETA: 47s - loss: 0.1040 - acc: 0.95 - ETA: 46s - loss: 0.1042 - acc: 0.95 - ETA: 46s - loss: 0.1039 - acc: 0.95 - ETA: 46s - loss: 0.1041 - acc: 0.95 - ETA: 45s - loss: 0.1038 - acc: 0.95 - ETA: 45s - loss: 0.1045 - acc: 0.95 - ETA: 45s - loss: 0.1043 - acc: 0.95 - ETA: 44s - loss: 0.1044 - acc: 0.95 - ETA: 44s - loss: 0.1045 - acc: 0.95 - ETA: 44s - loss: 0.1042 - acc: 0.95 - ETA: 43s - loss: 0.1043 - acc: 0.95 - E

323429/323429 [==============================] - ETA: 51s - loss: 0.0972 - acc: 0.96 - ETA: 52s - loss: 0.0938 - acc: 0.96 - ETA: 51s - loss: 0.0967 - acc: 0.96 - ETA: 51s - loss: 0.0947 - acc: 0.96 - ETA: 50s - loss: 0.0936 - acc: 0.96 - ETA: 50s - loss: 0.0951 - acc: 0.96 - ETA: 50s - loss: 0.0946 - acc: 0.96 - ETA: 49s - loss: 0.0934 - acc: 0.96 - ETA: 49s - loss: 0.0931 - acc: 0.96 - ETA: 49s - loss: 0.0937 - acc: 0.96 - ETA: 48s - loss: 0.0938 - acc: 0.96 - ETA: 48s - loss: 0.0933 - acc: 0.96 - ETA: 48s - loss: 0.0945 - acc: 0.96 - ETA: 47s - loss: 0.0934 - acc: 0.96 - ETA: 47s - loss: 0.0935 - acc: 0.96 - ETA: 47s - loss: 0.0931 - acc: 0.96 - ETA: 46s - loss: 0.0931 - acc: 0.96 - ETA: 46s - loss: 0.0930 - acc: 0.96 - ETA: 46s - loss: 0.0931 - acc: 0.96 - ETA: 45s - loss: 0.0927 - acc: 0.96 - ETA: 45s - loss: 0.0927 - acc: 0.96 - ETA: 45s - loss: 0.0924 - acc: 0.96 - ETA: 44s - loss: 0.0925 - acc: 0.96 - ETA: 44s - loss: 0.0928 - acc: 0.96 - ETA: 44s - loss: 0.0923 - acc: 0.96 - E

323429/323429 [==============================] - ETA: 54s - loss: 0.0869 - acc: 0.96 - ETA: 52s - loss: 0.0869 - acc: 0.96 - ETA: 51s - loss: 0.0857 - acc: 0.96 - ETA: 51s - loss: 0.0867 - acc: 0.96 - ETA: 51s - loss: 0.0847 - acc: 0.96 - ETA: 50s - loss: 0.0842 - acc: 0.96 - ETA: 50s - loss: 0.0813 - acc: 0.96 - ETA: 50s - loss: 0.0800 - acc: 0.96 - ETA: 49s - loss: 0.0804 - acc: 0.96 - ETA: 49s - loss: 0.0799 - acc: 0.96 - ETA: 48s - loss: 0.0810 - acc: 0.96 - ETA: 48s - loss: 0.0803 - acc: 0.96 - ETA: 48s - loss: 0.0803 - acc: 0.96 - ETA: 47s - loss: 0.0801 - acc: 0.96 - ETA: 47s - loss: 0.0802 - acc: 0.96 - ETA: 47s - loss: 0.0802 - acc: 0.96 - ETA: 46s - loss: 0.0798 - acc: 0.96 - ETA: 46s - loss: 0.0799 - acc: 0.96 - ETA: 46s - loss: 0.0806 - acc: 0.96 - ETA: 45s - loss: 0.0803 - acc: 0.96 - ETA: 45s - loss: 0.0806 - acc: 0.96 - ETA: 45s - loss: 0.0807 - acc: 0.96 - ETA: 44s - loss: 0.0810 - acc: 0.96 - ETA: 44s - loss: 0.0812 - acc: 0.96 - ETA: 44s - loss: 0.0816 - acc: 0.96 - E

323429/323429 [==============================] - ETA: 54s - loss: 0.0653 - acc: 0.97 - ETA: 52s - loss: 0.0658 - acc: 0.97 - ETA: 52s - loss: 0.0659 - acc: 0.97 - ETA: 51s - loss: 0.0651 - acc: 0.97 - ETA: 51s - loss: 0.0652 - acc: 0.97 - ETA: 50s - loss: 0.0659 - acc: 0.97 - ETA: 50s - loss: 0.0651 - acc: 0.97 - ETA: 50s - loss: 0.0675 - acc: 0.97 - ETA: 49s - loss: 0.0684 - acc: 0.97 - ETA: 49s - loss: 0.0689 - acc: 0.97 - ETA: 49s - loss: 0.0701 - acc: 0.97 - ETA: 48s - loss: 0.0698 - acc: 0.97 - ETA: 48s - loss: 0.0693 - acc: 0.97 - ETA: 48s - loss: 0.0693 - acc: 0.97 - ETA: 47s - loss: 0.0692 - acc: 0.97 - ETA: 47s - loss: 0.0694 - acc: 0.97 - ETA: 47s - loss: 0.0700 - acc: 0.97 - ETA: 46s - loss: 0.0698 - acc: 0.97 - ETA: 46s - loss: 0.0701 - acc: 0.97 - ETA: 46s - loss: 0.0697 - acc: 0.97 - ETA: 45s - loss: 0.0699 - acc: 0.97 - ETA: 45s - loss: 0.0712 - acc: 0.97 - ETA: 45s - loss: 0.0715 - acc: 0.97 - ETA: 44s - loss: 0.0715 - acc: 0.97 - ETA: 44s - loss: 0.0718 - acc: 0.97 - E

323429/323429 [==============================] - ETA: 54s - loss: 0.0479 - acc: 0.98 - ETA: 52s - loss: 0.0543 - acc: 0.97 - ETA: 52s - loss: 0.0594 - acc: 0.97 - ETA: 51s - loss: 0.0613 - acc: 0.97 - ETA: 50s - loss: 0.0627 - acc: 0.97 - ETA: 50s - loss: 0.0660 - acc: 0.97 - ETA: 50s - loss: 0.0663 - acc: 0.97 - ETA: 50s - loss: 0.0658 - acc: 0.97 - ETA: 49s - loss: 0.0667 - acc: 0.97 - ETA: 49s - loss: 0.0666 - acc: 0.97 - ETA: 49s - loss: 0.0665 - acc: 0.97 - ETA: 48s - loss: 0.0656 - acc: 0.97 - ETA: 48s - loss: 0.0655 - acc: 0.97 - ETA: 47s - loss: 0.0657 - acc: 0.97 - ETA: 47s - loss: 0.0657 - acc: 0.97 - ETA: 47s - loss: 0.0659 - acc: 0.97 - ETA: 46s - loss: 0.0658 - acc: 0.97 - ETA: 46s - loss: 0.0659 - acc: 0.97 - ETA: 46s - loss: 0.0655 - acc: 0.97 - ETA: 45s - loss: 0.0654 - acc: 0.97 - ETA: 45s - loss: 0.0656 - acc: 0.97 - ETA: 45s - loss: 0.0652 - acc: 0.97 - ETA: 44s - loss: 0.0656 - acc: 0.97 - ETA: 44s - loss: 0.0661 - acc: 0.97 - ETA: 44s - loss: 0.0658 - acc: 0.97 - E

### Siamese LTSM Experiment 2 (Just GloVe vectors)

#### Note : This program is almost similar to the one above but with some validation.

In [11]:
#siamese parameters
in_dim = len(tokenizr.word_index) + 1
out_dim = 300
_len = 30
_units = 250
_drop = 0.1

In [5]:
embed_layer = Embedding(in_dim, out_dim, weights=[embed_mat], input_length=_len)
lstm = LSTM(_units, dropout=_drop, recurrent_dropout=_drop)

#input sequences
question_1 = Input(shape=(30,), dtype="int32")
question_2 = Input(shape=(30,), dtype="int32")


#embedding the two input sequences
emb_question_1 = embed_layer(question_1)
emb_question_2 = embed_layer(question_2)


#create representations
_question_1 = lstm(emb_question_1)
_question_2 = lstm(emb_question_2)


_merge = concatenate([_question_1, _question_2])
_merge = Dense(150, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(150, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
is_duplicate = Dense(1, activation='sigmoid')(_merge)


model = Model(inputs=[question_1, question_2, features], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


fit = model.fit([train_q1, train_q2, list_of_all_training_features], train_labels, epochs=50, validation_split=0.1, batch_size=2000, shuffle=True)

# model.save('quora_model_1.h5')
# print("\n\n")
# print("LOSS : ", score[0])
# print("Accuracy : ", score[1]*100)
# print("\n\n")


predictions = model.predict([test_q1, test_q2, list_of_all_test_features], batch_size=8192, verbose=1)

submission = panda.DataFrame({'test_id':test_id, 'is_duplicate':predictions.ravel()})
submission.to_csv('result3.csv', index=False)

### Simple LSTM Experiment 2 - using just 20 features

In [12]:
#This LSTM was run using 20 features as input computed during the feature engineering phase.
#By implementing this code it was possible to save weights and stop training when there was no improvement 
#in the loss.

features = Input(shape=(20,), dtype='float32', name='main_input')
x = Embedding(output_dim=300, input_dim=10000, input_length=20)(features)
lstm_out = LSTM(lstm_units)(x)

_merge = Dense(300, activation='relu')(lstm_out)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
is_duplicate = Dense(1, activation='sigmoid')(_merge)


model = Model(inputs=[features], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

_stop =EarlyStopping(monitor='val_loss', patience=5)

_checkpoint = ModelCheckpoint('mymodel_250_1125_041217.h5', save_best_only=True, save_weights_only=True)

fit = model.fit([list_of_all_training_features], list_of_train_labels, epochs=100, verbose=1, validation_split=0.1, batch_size=384, shuffle=True, callbacks=[_stop, _checkpoint])


model.load_weights('mymodel_250_1125_041217.h5')


predictions = model.predict([list_of_all_test_features], batch_size=8192, verbose=1)

submission = panda.DataFrame({'test_id':test_id, 'is_duplicate':predictions.ravel()})
submission.to_csv('result6.csv', index=False)

Train on 363861 samples, validate on 40429 samples
Epoch 1/100
142848/363861 [==========>...................] - ETA: 36:03 - loss: 0.8898 - acc: 0.49 - ETA: 18:25 - loss: 0.8371 - acc: 0.54 - ETA: 12:32 - loss: 0.8079 - acc: 0.56 - ETA: 9:36 - loss: 0.7762 - acc: 0.5853 - ETA: 7:50 - loss: 0.7728 - acc: 0.592 - ETA: 5:47 - loss: 0.7545 - acc: 0.610 - ETA: 4:39 - loss: 0.7290 - acc: 0.627 - ETA: 3:55 - loss: 0.7090 - acc: 0.639 - ETA: 3:25 - loss: 0.7040 - acc: 0.645 - ETA: 3:03 - loss: 0.6873 - acc: 0.656 - ETA: 2:46 - loss: 0.6644 - acc: 0.667 - ETA: 2:32 - loss: 0.6532 - acc: 0.675 - ETA: 2:21 - loss: 0.6432 - acc: 0.681 - ETA: 2:17 - loss: 0.6416 - acc: 0.682 - ETA: 2:13 - loss: 0.6357 - acc: 0.685 - ETA: 2:05 - loss: 0.6261 - acc: 0.691 - ETA: 1:59 - loss: 0.6164 - acc: 0.698 - ETA: 1:53 - loss: 0.6071 - acc: 0.703 - ETA: 1:48 - loss: 0.5990 - acc: 0.706 - ETA: 1:44 - loss: 0.5916 - acc: 0.710 - ETA: 1:40 - loss: 0.5849 - acc: 0.712 - ETA: 1:37 - loss: 0.5819 - acc: 0.714 - ETA: 1:

152832/363861 [===========>..................] - ETA: 51s - loss: 0.2958 - acc: 0.86 - ETA: 45s - loss: 0.3073 - acc: 0.86 - ETA: 44s - loss: 0.3089 - acc: 0.86 - ETA: 43s - loss: 0.3151 - acc: 0.85 - ETA: 43s - loss: 0.3174 - acc: 0.85 - ETA: 43s - loss: 0.3146 - acc: 0.85 - ETA: 43s - loss: 0.3131 - acc: 0.85 - ETA: 42s - loss: 0.3130 - acc: 0.85 - ETA: 42s - loss: 0.3145 - acc: 0.85 - ETA: 42s - loss: 0.3123 - acc: 0.85 - ETA: 42s - loss: 0.3123 - acc: 0.85 - ETA: 41s - loss: 0.3119 - acc: 0.85 - ETA: 41s - loss: 0.3127 - acc: 0.85 - ETA: 41s - loss: 0.3105 - acc: 0.85 - ETA: 41s - loss: 0.3109 - acc: 0.85 - ETA: 41s - loss: 0.3102 - acc: 0.85 - ETA: 41s - loss: 0.3105 - acc: 0.85 - ETA: 41s - loss: 0.3099 - acc: 0.85 - ETA: 41s - loss: 0.3087 - acc: 0.85 - ETA: 41s - loss: 0.3091 - acc: 0.85 - ETA: 41s - loss: 0.3094 - acc: 0.85 - ETA: 41s - loss: 0.3081 - acc: 0.85 - ETA: 40s - loss: 0.3092 - acc: 0.85 - ETA: 40s - loss: 0.3092 - acc: 0.85 - ETA: 40s - loss: 0.3104 - acc: 0.85 - E

363861/363861 [==============================] - ETA: 6s - loss: 0.3084 - acc: 0.858 - ETA: 6s - loss: 0.3083 - acc: 0.858 - ETA: 6s - loss: 0.3083 - acc: 0.858 - ETA: 5s - loss: 0.3082 - acc: 0.858 - ETA: 5s - loss: 0.3082 - acc: 0.858 - ETA: 5s - loss: 0.3081 - acc: 0.858 - ETA: 5s - loss: 0.3081 - acc: 0.858 - ETA: 5s - loss: 0.3081 - acc: 0.858 - ETA: 5s - loss: 0.3081 - acc: 0.858 - ETA: 5s - loss: 0.3082 - acc: 0.858 - ETA: 5s - loss: 0.3082 - acc: 0.858 - ETA: 5s - loss: 0.3083 - acc: 0.858 - ETA: 5s - loss: 0.3082 - acc: 0.858 - ETA: 5s - loss: 0.3083 - acc: 0.858 - ETA: 4s - loss: 0.3082 - acc: 0.858 - ETA: 4s - loss: 0.3082 - acc: 0.858 - ETA: 4s - loss: 0.3082 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - ETA: 4s - loss: 0.3081 - acc: 0.858 - E

155904/363861 [===========>..................] - ETA: 1:07 - loss: 0.2951 - acc: 0.856 - ETA: 49s - loss: 0.2791 - acc: 0.873 - ETA: 46s - loss: 0.2921 - acc: 0.87 - ETA: 44s - loss: 0.2970 - acc: 0.86 - ETA: 43s - loss: 0.2965 - acc: 0.86 - ETA: 43s - loss: 0.2951 - acc: 0.86 - ETA: 42s - loss: 0.2940 - acc: 0.86 - ETA: 42s - loss: 0.2896 - acc: 0.86 - ETA: 42s - loss: 0.2859 - acc: 0.86 - ETA: 42s - loss: 0.2905 - acc: 0.86 - ETA: 41s - loss: 0.2915 - acc: 0.86 - ETA: 42s - loss: 0.2923 - acc: 0.86 - ETA: 41s - loss: 0.2920 - acc: 0.86 - ETA: 41s - loss: 0.2927 - acc: 0.86 - ETA: 41s - loss: 0.2918 - acc: 0.86 - ETA: 41s - loss: 0.2932 - acc: 0.86 - ETA: 41s - loss: 0.2926 - acc: 0.86 - ETA: 41s - loss: 0.2928 - acc: 0.86 - ETA: 41s - loss: 0.2926 - acc: 0.86 - ETA: 40s - loss: 0.2918 - acc: 0.86 - ETA: 40s - loss: 0.2915 - acc: 0.86 - ETA: 40s - loss: 0.2920 - acc: 0.86 - ETA: 40s - loss: 0.2921 - acc: 0.86 - ETA: 40s - loss: 0.2931 - acc: 0.86 - ETA: 40s - loss: 0.2952 - acc: 0.86 

363861/363861 [==============================] - ETA: 6s - loss: 0.2970 - acc: 0.864 - ETA: 6s - loss: 0.2970 - acc: 0.864 - ETA: 5s - loss: 0.2970 - acc: 0.864 - ETA: 5s - loss: 0.2970 - acc: 0.864 - ETA: 5s - loss: 0.2970 - acc: 0.864 - ETA: 5s - loss: 0.2971 - acc: 0.864 - ETA: 5s - loss: 0.2971 - acc: 0.864 - ETA: 5s - loss: 0.2971 - acc: 0.864 - ETA: 5s - loss: 0.2972 - acc: 0.864 - ETA: 5s - loss: 0.2972 - acc: 0.864 - ETA: 5s - loss: 0.2972 - acc: 0.864 - ETA: 5s - loss: 0.2972 - acc: 0.864 - ETA: 5s - loss: 0.2972 - acc: 0.864 - ETA: 5s - loss: 0.2972 - acc: 0.864 - ETA: 4s - loss: 0.2972 - acc: 0.864 - ETA: 4s - loss: 0.2972 - acc: 0.864 - ETA: 4s - loss: 0.2973 - acc: 0.864 - ETA: 4s - loss: 0.2973 - acc: 0.864 - ETA: 4s - loss: 0.2973 - acc: 0.864 - ETA: 4s - loss: 0.2973 - acc: 0.864 - ETA: 4s - loss: 0.2972 - acc: 0.864 - ETA: 4s - loss: 0.2972 - acc: 0.864 - ETA: 4s - loss: 0.2971 - acc: 0.864 - ETA: 4s - loss: 0.2971 - acc: 0.864 - ETA: 4s - loss: 0.2970 - acc: 0.864 - E

...

363861/363861 [==============================] - ETA: 6s - loss: 0.2853 - acc: 0.870 - ETA: 6s - loss: 0.2853 - acc: 0.870 - ETA: 6s - loss: 0.2853 - acc: 0.870 - ETA: 6s - loss: 0.2854 - acc: 0.870 - ETA: 6s - loss: 0.2853 - acc: 0.870 - ETA: 6s - loss: 0.2853 - acc: 0.870 - ETA: 6s - loss: 0.2853 - acc: 0.870 - ETA: 6s - loss: 0.2852 - acc: 0.870 - ETA: 6s - loss: 0.2852 - acc: 0.870 - ETA: 5s - loss: 0.2852 - acc: 0.870 - ETA: 5s - loss: 0.2853 - acc: 0.870 - ETA: 5s - loss: 0.2853 - acc: 0.870 - ETA: 5s - loss: 0.2853 - acc: 0.870 - ETA: 5s - loss: 0.2853 - acc: 0.870 - ETA: 5s - loss: 0.2853 - acc: 0.870 - ETA: 5s - loss: 0.2853 - acc: 0.870 - ETA: 5s - loss: 0.2852 - acc: 0.870 - ETA: 5s - loss: 0.2851 - acc: 0.870 - ETA: 5s - loss: 0.2851 - acc: 0.870 - ETA: 5s - loss: 0.2851 - acc: 0.870 - ETA: 4s - loss: 0.2851 - acc: 0.870 - ETA: 4s - loss: 0.2851 - acc: 0.870 - ETA: 4s - loss: 0.2851 - acc: 0.870 - ETA: 4s - loss: 0.2851 - acc: 0.870 - ETA: 4s - loss: 0.2851 - acc: 0.870 - E

155904/363861 [===========>..................] - ETA: 49s - loss: 0.2937 - acc: 0.87 - ETA: 44s - loss: 0.2528 - acc: 0.88 - ETA: 43s - loss: 0.2556 - acc: 0.88 - ETA: 42s - loss: 0.2545 - acc: 0.88 - ETA: 42s - loss: 0.2570 - acc: 0.88 - ETA: 42s - loss: 0.2623 - acc: 0.88 - ETA: 41s - loss: 0.2660 - acc: 0.87 - ETA: 41s - loss: 0.2674 - acc: 0.88 - ETA: 41s - loss: 0.2663 - acc: 0.88 - ETA: 41s - loss: 0.2688 - acc: 0.88 - ETA: 41s - loss: 0.2676 - acc: 0.88 - ETA: 41s - loss: 0.2704 - acc: 0.87 - ETA: 41s - loss: 0.2710 - acc: 0.87 - ETA: 41s - loss: 0.2702 - acc: 0.87 - ETA: 41s - loss: 0.2731 - acc: 0.87 - ETA: 41s - loss: 0.2727 - acc: 0.87 - ETA: 41s - loss: 0.2729 - acc: 0.87 - ETA: 41s - loss: 0.2735 - acc: 0.87 - ETA: 41s - loss: 0.2723 - acc: 0.87 - ETA: 40s - loss: 0.2728 - acc: 0.87 - ETA: 40s - loss: 0.2733 - acc: 0.87 - ETA: 40s - loss: 0.2740 - acc: 0.87 - ETA: 40s - loss: 0.2744 - acc: 0.87 - ETA: 40s - loss: 0.2747 - acc: 0.87 - ETA: 40s - loss: 0.2741 - acc: 0.87 - E

363861/363861 [==============================] - ETA: 6s - loss: 0.2814 - acc: 0.872 - ETA: 6s - loss: 0.2814 - acc: 0.872 - ETA: 6s - loss: 0.2813 - acc: 0.872 - ETA: 5s - loss: 0.2813 - acc: 0.872 - ETA: 5s - loss: 0.2814 - acc: 0.872 - ETA: 5s - loss: 0.2814 - acc: 0.872 - ETA: 5s - loss: 0.2814 - acc: 0.872 - ETA: 5s - loss: 0.2814 - acc: 0.872 - ETA: 5s - loss: 0.2814 - acc: 0.872 - ETA: 5s - loss: 0.2813 - acc: 0.872 - ETA: 5s - loss: 0.2813 - acc: 0.872 - ETA: 5s - loss: 0.2813 - acc: 0.872 - ETA: 5s - loss: 0.2812 - acc: 0.872 - ETA: 5s - loss: 0.2812 - acc: 0.872 - ETA: 4s - loss: 0.2812 - acc: 0.872 - ETA: 4s - loss: 0.2812 - acc: 0.872 - ETA: 4s - loss: 0.2812 - acc: 0.872 - ETA: 4s - loss: 0.2811 - acc: 0.872 - ETA: 4s - loss: 0.2812 - acc: 0.872 - ETA: 4s - loss: 0.2812 - acc: 0.872 - ETA: 4s - loss: 0.2812 - acc: 0.872 - ETA: 4s - loss: 0.2811 - acc: 0.872 - ETA: 4s - loss: 0.2811 - acc: 0.872 - ETA: 4s - loss: 0.2811 - acc: 0.872 - ETA: 4s - loss: 0.2810 - acc: 0.872 - E

163584/363861 [============>.................] - ETA: 48s - loss: 0.2877 - acc: 0.85 - ETA: 43s - loss: 0.2677 - acc: 0.86 - ETA: 42s - loss: 0.2770 - acc: 0.87 - ETA: 42s - loss: 0.2854 - acc: 0.86 - ETA: 41s - loss: 0.2811 - acc: 0.86 - ETA: 41s - loss: 0.2803 - acc: 0.87 - ETA: 41s - loss: 0.2808 - acc: 0.87 - ETA: 41s - loss: 0.2808 - acc: 0.87 - ETA: 40s - loss: 0.2807 - acc: 0.87 - ETA: 40s - loss: 0.2773 - acc: 0.87 - ETA: 40s - loss: 0.2757 - acc: 0.87 - ETA: 40s - loss: 0.2738 - acc: 0.87 - ETA: 40s - loss: 0.2746 - acc: 0.87 - ETA: 40s - loss: 0.2741 - acc: 0.87 - ETA: 40s - loss: 0.2741 - acc: 0.87 - ETA: 39s - loss: 0.2756 - acc: 0.87 - ETA: 39s - loss: 0.2733 - acc: 0.87 - ETA: 39s - loss: 0.2738 - acc: 0.87 - ETA: 39s - loss: 0.2725 - acc: 0.87 - ETA: 39s - loss: 0.2729 - acc: 0.87 - ETA: 39s - loss: 0.2728 - acc: 0.87 - ETA: 39s - loss: 0.2708 - acc: 0.87 - ETA: 39s - loss: 0.2702 - acc: 0.87 - ETA: 39s - loss: 0.2694 - acc: 0.87 - ETA: 39s - loss: 0.2705 - acc: 0.87 - E

2345796/2345796 [==============================] - ETA: 1: - ETA: 1: - ETA: 55s - ETA: 48 - ETA: 44 - ETA: 42 - ETA: 40 - ETA: 39 - ETA: 38 - ETA: 37 - ETA: 36 - ETA: 35 - ETA: 35 - ETA: 34 - ETA: 34 - ETA: 33 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 29 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 28 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 26 - ETA: 25 - ETA: 25 - ETA: 25 - ETA: 25 - ETA: 25 - ETA: 25 - ETA: 25 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 

### Simple LSTM Experiment 3 - Denser Network

In [24]:
#A similar experiment as above. But with a more denser fully connected layers.

features = Input(shape=(20,), dtype='float32', name='main_input')
x = Embedding(output_dim=300, input_dim=10000, input_length=20)(features)
lstm_out = LSTM(lstm_units)(x)

#added more dense layers
_merge = Dense(200, activation='relu')(lstm_out)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(200, activation='relu')(_merge)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(200, activation='relu')(_merge)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(200, activation='relu')(_merge)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(200, activation='relu')(_merge)
_merge = Dropout(drop)(_merge)
_merge = BatchNormalization()(_merge)
is_duplicate = Dense(1, activation='sigmoid')(_merge)


model = Model(inputs=[features], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

_stop =EarlyStopping(monitor='val_loss', patience=6)

_checkpoint = ModelCheckpoint('mymodel_dense_250_1151_041217.h5', save_best_only=True, save_weights_only=True)

fit = model.fit([list_of_all_training_features], list_of_train_labels, epochs=200, verbose=1, validation_split=0.1, batch_size=384, shuffle=True, callbacks=[_stop, _checkpoint])


model.load_weights('mymodel_dense_250_1151_041217.h5')


predictions = model.predict([list_of_all_test_features], batch_size=8192, verbose=1)

submission = panda.DataFrame({'test_id':test_id, 'is_duplicate':predictions.ravel()})
submission.to_csv('result8.csv', index=False)

Train on 363861 samples, validate on 40429 samples
Epoch 1/200


KeyboardInterrupt: 

### Back to Siamese - Experiment 3 - A much denser Siamese Network

In [20]:
in_dim = len(tokenizr.word_index) + 1
out_dim = 300
_len = 30
_units = 250
_drop = 0.4

In [6]:
#Used more dense layers to test.

embed_layer = Embedding(in_dim, out_dim, weights=[embed_mat], input_length=_len)
lstm = LSTM(_units, dropout=_drop, recurrent_dropout=_drop)

#input sequences
question_1 = Input(shape=(30,), dtype="int32")
question_2 = Input(shape=(30,), dtype="int32")


#embedding the two input sequences
emb_question_1 = embed_layer(question_1)
emb_question_2 = embed_layer(question_2)


#create representations
_question_1 = lstm(emb_question_1)
_question_2 = lstm(emb_question_2)


_merge = concatenate([_question_1, _question_2])
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
is_duplicate = Dense(1, activation='sigmoid')(_merge)


model = Model(inputs=[question_1, question_2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

_stop =EarlyStopping(monitor='val_loss', patience=6)
_checkpoint = ModelCheckpoint('dense_siamese_lstm_250_342_051217.h5', save_best_only=True, save_weights_only=True)
fit = model.fit([train_q1, train_q2], train_labels, epochs=200, validation_split=0.1, batch_size=2000, shuffle=True, callbacks=[_stop, _checkpoint])

# model.save('quora_model_1.h5')
# print("\n\n")
# print("LOSS : ", score[0])
# print("Accuracy : ", score[1]*100)
# print("\n\n")


model.load_weights('dense_siamese_lstm_250_342_051217.h5')
predictions = model.predict([test_q1, test_q2], batch_size=8192, verbose=1)

submission = panda.DataFrame({'test_id':test_id, 'is_duplicate':predictions.ravel()})
submission.to_csv('result9.csv', index=False)

NameError: name 'Embedding' is not defined

###         

### A Slightly different architecture using the Siamese LSTM


#### After many experiments, finally found that this architecture gives a good log loss score on Kaggle
#### This gave around 0.181 score for the log loss.

In [45]:
input_dim = len(tokenizr.word_index) + 1
output_dim = 300
input_len = 60
lstm_units = 250
drop = 0.4
in_dim = len(tokenizr.word_index) + 1
out_dim = 300
_len = 30
_units = 250
_drop = 0.4

In [8]:
#embedding layer for sentence vectors
embed_layer = Embedding(in_dim, out_dim, weights=[embed_mat], input_length=_len)
lstm = LSTM(_units, dropout=_drop, recurrent_dropout=_drop)


#getting the features
features = Input(shape=(20,), dtype='float32', name='main_input')
x = Embedding(output_dim=300, input_dim=10000, input_length=20)(features)
lstm_out = LSTM(lstm_units)(x)


#input sequences
question_1 = Input(shape=(30,), dtype="int32")
question_2 = Input(shape=(30,), dtype="int32")


#embedding the two input sequences
emb_question_1 = embed_layer(question_1)
emb_question_2 = embed_layer(question_2)


#create representations
_question_1 = lstm(emb_question_1)
_question_2 = lstm(emb_question_2)


#combining features and sentence vectors
_merge = concatenate([_question_1, _question_2, lstm_out])
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
is_duplicate = Dense(1, activation='sigmoid')(_merge)


model = Model(inputs=[question_1, question_2, features], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

_stop =EarlyStopping(monitor='val_loss', patience=6)
_checkpoint = ModelCheckpoint('feature_siamese_lstm_250_543_071217.h5', save_best_only=True, save_weights_only=True)
fit = model.fit([train_q1, train_q2, list_of_all_training_features], train_labels, epochs=200, validation_split=0.1, batch_size=2000, shuffle=True, callbacks=[_stop, _checkpoint])

# model.save('quora_model_1.h5')
# print("\n\n")
# print("LOSS : ", score[0])
# print("Accuracy : ", score[1]*100)
# print("\n\n")


model.load_weights('feature_siamese_lstm_250_543_071217.h5')
predictions = model.predict([test_q1, test_q2, list_of_all_test_features], batch_size=8192, verbose=1)

submission = panda.DataFrame({'test_id':test_id, 'is_duplicate':predictions.ravel()})
submission.to_csv('result13.csv', index=False)

NameError: name 'Embedding' is not defined

###       

### Bidirectional Siamese LSTM
#### An experiment to see how this network performs
#### Got a log loss of 0.22 on Kaggle. Not a good result.

In [29]:
input_dim = len(tokenizr.word_index) + 1
output_dim = 300
input_len = 60
lstm_units = 250
drop = 0.4
in_dim = len(tokenizr.word_index) + 1
out_dim = 300
_len = 30
_units = 250
_drop = 0.4

In [30]:
embed_layer = Embedding(in_dim, out_dim, weights=[embed_mat], input_length=_len)
lstm = Bidirectional(LSTM(_units, dropout=_drop, recurrent_dropout=_drop))

features = Input(shape=(20,), dtype='float32', name='main_input')
x = Embedding(output_dim=300, input_dim=10000, input_length=20)(features)
lstm_out = Bidirectional(LSTM(lstm_units))(x)

#input sequences
question_1 = Input(shape=(30,), dtype="int32")
question_2 = Input(shape=(30,), dtype="int32")


#embedding the two input sequences
emb_question_1 = embed_layer(question_1)
emb_question_2 = embed_layer(question_2)


#create representations
_question_1 = lstm(emb_question_1)
_question_2 = lstm(emb_question_2)


_merge = concatenate([_question_1, _question_2, lstm_out])
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
_merge = Dense(300, activation='relu')(_merge)
_merge = Dropout(_drop)(_merge)
_merge = BatchNormalization()(_merge)
is_duplicate = Dense(1, activation='sigmoid')(_merge)


model = Model(inputs=[question_1, question_2, features], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

_stop =EarlyStopping(monitor='val_loss', patience=6)
_checkpoint = ModelCheckpoint('feature_siamese_bilstm_250_219_071217.h5', save_best_only=True, save_weights_only=True)
fit = model.fit([train_q1, train_q2, list_of_all_training_features], train_labels, epochs=200, validation_split=0.1, batch_size=2000, shuffle=True, callbacks=[_stop, _checkpoint])

# model.save('quora_model_1.h5')
# print("\n\n")
# print("LOSS : ", score[0])
# print("Accuracy : ", score[1]*100)
# print("\n\n")


model.load_weights('feature_siamese_bilstm_250_219_071217.h5')
predictions = model.predict([test_q1, test_q2, list_of_all_test_features], batch_size=8192, verbose=1)

submission = panda.DataFrame({'test_id':test_id, 'is_duplicate':predictions.ravel()})
submission.to_csv('result11.csv', index=False)

Train on 363861 samples, validate on 40429 samples
Epoch 1/200
363861/363861 [==============================] - ETA: 30:20 - loss: 0.9192 - acc: 0.48 - ETA: 17:34 - loss: 0.8994 - acc: 0.49 - ETA: 13:19 - loss: 0.8811 - acc: 0.49 - ETA: 11:08 - loss: 0.8733 - acc: 0.50 - ETA: 9:51 - loss: 0.8683 - acc: 0.5025 - ETA: 8:57 - loss: 0.8592 - acc: 0.503 - ETA: 8:20 - loss: 0.8536 - acc: 0.503 - ETA: 7:52 - loss: 0.8481 - acc: 0.504 - ETA: 7:28 - loss: 0.8406 - acc: 0.505 - ETA: 7:09 - loss: 0.8310 - acc: 0.509 - ETA: 6:54 - loss: 0.8249 - acc: 0.512 - ETA: 6:41 - loss: 0.8183 - acc: 0.515 - ETA: 6:29 - loss: 0.8118 - acc: 0.517 - ETA: 6:19 - loss: 0.8083 - acc: 0.519 - ETA: 6:10 - loss: 0.8018 - acc: 0.523 - ETA: 6:02 - loss: 0.7951 - acc: 0.528 - ETA: 5:55 - loss: 0.7872 - acc: 0.533 - ETA: 5:48 - loss: 0.7806 - acc: 0.538 - ETA: 5:42 - loss: 0.7723 - acc: 0.544 - ETA: 5:36 - loss: 0.7640 - acc: 0.550 - ETA: 5:31 - loss: 0.7574 - acc: 0.555 - ETA: 5:26 - loss: 0.7509 - acc: 0.559 - ETA: 5:

363861/363861 [==============================] - ETA: 5:13 - loss: 0.2768 - acc: 0.870 - ETA: 5:09 - loss: 0.2784 - acc: 0.875 - ETA: 5:04 - loss: 0.2794 - acc: 0.873 - ETA: 5:01 - loss: 0.2822 - acc: 0.872 - ETA: 4:56 - loss: 0.2821 - acc: 0.872 - ETA: 4:55 - loss: 0.2816 - acc: 0.873 - ETA: 4:51 - loss: 0.2824 - acc: 0.873 - ETA: 4:50 - loss: 0.2834 - acc: 0.872 - ETA: 4:48 - loss: 0.2836 - acc: 0.872 - ETA: 4:46 - loss: 0.2832 - acc: 0.872 - ETA: 4:44 - loss: 0.2832 - acc: 0.872 - ETA: 4:42 - loss: 0.2831 - acc: 0.872 - ETA: 4:40 - loss: 0.2833 - acc: 0.872 - ETA: 4:39 - loss: 0.2834 - acc: 0.871 - ETA: 4:37 - loss: 0.2822 - acc: 0.872 - ETA: 4:35 - loss: 0.2820 - acc: 0.872 - ETA: 4:33 - loss: 0.2816 - acc: 0.872 - ETA: 4:32 - loss: 0.2816 - acc: 0.872 - ETA: 4:30 - loss: 0.2826 - acc: 0.872 - ETA: 4:28 - loss: 0.2838 - acc: 0.871 - ETA: 4:27 - loss: 0.2846 - acc: 0.871 - ETA: 4:25 - loss: 0.2861 - acc: 0.871 - ETA: 4:24 - loss: 0.2849 - acc: 0.871 - ETA: 4:22 - loss: 0.2857 - acc:

363861/363861 [==============================] - ETA: 5:09 - loss: 0.2491 - acc: 0.888 - ETA: 4:57 - loss: 0.2613 - acc: 0.882 - ETA: 4:57 - loss: 0.2585 - acc: 0.884 - ETA: 4:51 - loss: 0.2583 - acc: 0.884 - ETA: 4:52 - loss: 0.2572 - acc: 0.885 - ETA: 4:50 - loss: 0.2577 - acc: 0.885 - ETA: 4:45 - loss: 0.2583 - acc: 0.885 - ETA: 4:43 - loss: 0.2593 - acc: 0.885 - ETA: 4:42 - loss: 0.2583 - acc: 0.885 - ETA: 4:40 - loss: 0.2580 - acc: 0.885 - ETA: 4:39 - loss: 0.2569 - acc: 0.886 - ETA: 4:37 - loss: 0.2564 - acc: 0.887 - ETA: 4:36 - loss: 0.2563 - acc: 0.886 - ETA: 4:33 - loss: 0.2563 - acc: 0.886 - ETA: 4:31 - loss: 0.2549 - acc: 0.886 - ETA: 4:30 - loss: 0.2549 - acc: 0.886 - ETA: 4:29 - loss: 0.2542 - acc: 0.886 - ETA: 4:28 - loss: 0.2543 - acc: 0.886 - ETA: 4:26 - loss: 0.2548 - acc: 0.886 - ETA: 4:25 - loss: 0.2539 - acc: 0.887 - ETA: 4:23 - loss: 0.2535 - acc: 0.887 - ETA: 4:21 - loss: 0.2530 - acc: 0.887 - ETA: 4:19 - loss: 0.2524 - acc: 0.887 - ETA: 4:18 - loss: 0.2522 - acc:

363861/363861 [==============================] - ETA: 5:02 - loss: 0.2043 - acc: 0.915 - ETA: 4:49 - loss: 0.2066 - acc: 0.910 - ETA: 4:50 - loss: 0.2105 - acc: 0.906 - ETA: 4:48 - loss: 0.2152 - acc: 0.903 - ETA: 4:48 - loss: 0.2107 - acc: 0.906 - ETA: 4:47 - loss: 0.2108 - acc: 0.907 - ETA: 4:46 - loss: 0.2123 - acc: 0.906 - ETA: 4:44 - loss: 0.2150 - acc: 0.905 - ETA: 4:41 - loss: 0.2168 - acc: 0.905 - ETA: 4:41 - loss: 0.2170 - acc: 0.904 - ETA: 4:40 - loss: 0.2178 - acc: 0.905 - ETA: 4:38 - loss: 0.2188 - acc: 0.904 - ETA: 4:37 - loss: 0.2193 - acc: 0.904 - ETA: 4:36 - loss: 0.2198 - acc: 0.903 - ETA: 4:34 - loss: 0.2201 - acc: 0.903 - ETA: 4:32 - loss: 0.2198 - acc: 0.903 - ETA: 4:30 - loss: 0.2194 - acc: 0.903 - ETA: 4:28 - loss: 0.2186 - acc: 0.904 - ETA: 4:26 - loss: 0.2188 - acc: 0.904 - ETA: 4:25 - loss: 0.2185 - acc: 0.904 - ETA: 4:23 - loss: 0.2189 - acc: 0.904 - ETA: 4:22 - loss: 0.2190 - acc: 0.903 - ETA: 4:20 - loss: 0.2184 - acc: 0.904 - ETA: 4:19 - loss: 0.2181 - acc:

363861/363861 [==============================] - ETA: 5:08 - loss: 0.1941 - acc: 0.917 - ETA: 5:03 - loss: 0.1937 - acc: 0.918 - ETA: 5:00 - loss: 0.1999 - acc: 0.914 - ETA: 4:58 - loss: 0.1964 - acc: 0.914 - ETA: 4:57 - loss: 0.1956 - acc: 0.914 - ETA: 4:55 - loss: 0.1963 - acc: 0.914 - ETA: 4:54 - loss: 0.1951 - acc: 0.915 - ETA: 4:52 - loss: 0.1949 - acc: 0.915 - ETA: 4:48 - loss: 0.1962 - acc: 0.914 - ETA: 4:47 - loss: 0.1976 - acc: 0.913 - ETA: 4:44 - loss: 0.1961 - acc: 0.914 - ETA: 4:43 - loss: 0.1961 - acc: 0.915 - ETA: 4:41 - loss: 0.1944 - acc: 0.916 - ETA: 4:39 - loss: 0.1945 - acc: 0.915 - ETA: 4:37 - loss: 0.1956 - acc: 0.915 - ETA: 4:34 - loss: 0.1943 - acc: 0.915 - ETA: 4:33 - loss: 0.1938 - acc: 0.916 - ETA: 4:30 - loss: 0.1926 - acc: 0.916 - ETA: 4:29 - loss: 0.1927 - acc: 0.916 - ETA: 4:28 - loss: 0.1938 - acc: 0.916 - ETA: 4:26 - loss: 0.1937 - acc: 0.916 - ETA: 4:24 - loss: 0.1941 - acc: 0.915 - ETA: 4:22 - loss: 0.1939 - acc: 0.915 - ETA: 4:21 - loss: 0.1937 - acc:

2345796/2345796 [==============================] - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: -

###   

### XGBoost
#### This alone gave a log loss of 0.17+ on Kaggle

In [31]:
pos_train = result_1[y_train == 1]
neg_train = result_1[y_train == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = panda.concat([neg_train, neg_train])
    scale -=1
neg_train = panda.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

x_train = panda.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train


# Finally, we split some of the data off for validation
# from sklearn.cross_validation import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)



# import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.01
params['max_depth'] = 4
params['min_child_weight'] = 6
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['alpha'] = 0.005 #default 0

params['seed'] = 27
# n_estimators=5000,

#params['scale_pos_weight'] = 1


d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 50000, watchlist, early_stopping_rounds=100, verbose_eval=10)





d_test = xgb.DMatrix(result_2)
p_test = bst.predict(d_test)

sub = panda.DataFrame()
sub['test_id'] = test_data['test_id']
sub['is_duplicate'] = p_test
sub.to_csv('20features_xgb_50k.csv', index=False)

0.19124366100096607
[0]	train-logloss:0.686128	valid-logloss:0.686151
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.62278	valid-logloss:0.623065
[20]	train-logloss:0.570144	valid-logloss:0.570651
[30]	train-logloss:0.525754	valid-logloss:0.526485
[40]	train-logloss:0.488858	valid-logloss:0.489792
[50]	train-logloss:0.456377	valid-logloss:0.457491
[60]	train-logloss:0.428695	valid-logloss:0.42998
[70]	train-logloss:0.404315	valid-logloss:0.405764
[80]	train-logloss:0.383566	valid-logloss:0.385159
[90]	train-logloss:0.365265	valid-logloss:0.366986
[100]	train-logloss:0.348919	valid-logloss:0.350773
[110]	train-logloss:0.334549	valid-logloss:0.33653
[120]	train-logloss:0.321685	valid-logloss:0.323798
[130]	train-logloss:0.310313	valid-logloss:0.312546
[140]	train-logloss:0.300273	valid-logloss:0.302608
[150]	train-logloss:0.291349	valid-logloss:0.293782
[160]	trai

[1540]	train-logloss:0.197161	valid-logloss:0.202066
[1550]	train-logloss:0.197114	valid-logloss:0.202026
[1560]	train-logloss:0.197067	valid-logloss:0.20199
[1570]	train-logloss:0.197035	valid-logloss:0.201965
[1580]	train-logloss:0.196997	valid-logloss:0.201936
[1590]	train-logloss:0.196961	valid-logloss:0.201906
[1600]	train-logloss:0.196919	valid-logloss:0.201874
[1610]	train-logloss:0.196876	valid-logloss:0.201838
[1620]	train-logloss:0.196831	valid-logloss:0.2018
[1630]	train-logloss:0.196792	valid-logloss:0.201766
[1640]	train-logloss:0.196747	valid-logloss:0.201732
[1650]	train-logloss:0.196711	valid-logloss:0.201701
[1660]	train-logloss:0.196663	valid-logloss:0.201658
[1670]	train-logloss:0.196624	valid-logloss:0.201625
[1680]	train-logloss:0.196592	valid-logloss:0.2016
[1690]	train-logloss:0.196549	valid-logloss:0.201565
[1700]	train-logloss:0.1965	valid-logloss:0.201523
[1710]	train-logloss:0.196449	valid-logloss:0.20148
[1720]	train-logloss:0.196406	valid-logloss:0.201446
[

[3100]	train-logloss:0.192631	valid-logloss:0.198647
[3110]	train-logloss:0.192606	valid-logloss:0.198629
[3120]	train-logloss:0.192583	valid-logloss:0.198613
[3130]	train-logloss:0.192558	valid-logloss:0.198595
[3140]	train-logloss:0.192542	valid-logloss:0.198584
[3150]	train-logloss:0.192522	valid-logloss:0.198573
[3160]	train-logloss:0.192502	valid-logloss:0.198561
[3170]	train-logloss:0.19248	valid-logloss:0.198545
[3180]	train-logloss:0.192458	valid-logloss:0.198533
[3190]	train-logloss:0.192444	valid-logloss:0.198526
[3200]	train-logloss:0.192422	valid-logloss:0.19851
[3210]	train-logloss:0.192405	valid-logloss:0.198501
[3220]	train-logloss:0.192386	valid-logloss:0.198486
[3230]	train-logloss:0.192368	valid-logloss:0.198474
[3240]	train-logloss:0.192348	valid-logloss:0.198462
[3250]	train-logloss:0.192326	valid-logloss:0.198446
[3260]	train-logloss:0.192308	valid-logloss:0.198435
[3270]	train-logloss:0.192293	valid-logloss:0.198425
[3280]	train-logloss:0.192277	valid-logloss:0.19

[4660]	train-logloss:0.190074	valid-logloss:0.197061
[4670]	train-logloss:0.19006	valid-logloss:0.197054
[4680]	train-logloss:0.190047	valid-logloss:0.197047
[4690]	train-logloss:0.190034	valid-logloss:0.197041
[4700]	train-logloss:0.19002	valid-logloss:0.197035
[4710]	train-logloss:0.190005	valid-logloss:0.197027
[4720]	train-logloss:0.189992	valid-logloss:0.197019
[4730]	train-logloss:0.189978	valid-logloss:0.197009
[4740]	train-logloss:0.189964	valid-logloss:0.197002
[4750]	train-logloss:0.189946	valid-logloss:0.196992
[4760]	train-logloss:0.189936	valid-logloss:0.196987
[4770]	train-logloss:0.189923	valid-logloss:0.196979
[4780]	train-logloss:0.189914	valid-logloss:0.196975
[4790]	train-logloss:0.1899	valid-logloss:0.196967
[4800]	train-logloss:0.189884	valid-logloss:0.196957
[4810]	train-logloss:0.18987	valid-logloss:0.196949
[4820]	train-logloss:0.189854	valid-logloss:0.196937
[4830]	train-logloss:0.189839	valid-logloss:0.196928
[4840]	train-logloss:0.18983	valid-logloss:0.196923

[6220]	train-logloss:0.188121	valid-logloss:0.196019
[6230]	train-logloss:0.188111	valid-logloss:0.196017
[6240]	train-logloss:0.188099	valid-logloss:0.19601
[6250]	train-logloss:0.188088	valid-logloss:0.196006
[6260]	train-logloss:0.188076	valid-logloss:0.196
[6270]	train-logloss:0.188066	valid-logloss:0.195996
[6280]	train-logloss:0.188056	valid-logloss:0.195992
[6290]	train-logloss:0.188045	valid-logloss:0.195987
[6300]	train-logloss:0.188033	valid-logloss:0.195981
[6310]	train-logloss:0.188018	valid-logloss:0.195972
[6320]	train-logloss:0.188007	valid-logloss:0.195966
[6330]	train-logloss:0.187996	valid-logloss:0.195961
[6340]	train-logloss:0.187988	valid-logloss:0.195959
[6350]	train-logloss:0.187978	valid-logloss:0.195954
[6360]	train-logloss:0.187966	valid-logloss:0.195946
[6370]	train-logloss:0.187953	valid-logloss:0.19594
[6380]	train-logloss:0.18794	valid-logloss:0.195932
[6390]	train-logloss:0.187926	valid-logloss:0.195925
[6400]	train-logloss:0.187914	valid-logloss:0.195919

...

[43010]	train-logloss:0.164876	valid-logloss:0.187695
[43020]	train-logloss:0.164871	valid-logloss:0.187692
[43030]	train-logloss:0.164867	valid-logloss:0.187692
[43040]	train-logloss:0.164863	valid-logloss:0.187691
[43050]	train-logloss:0.164859	valid-logloss:0.187689
[43060]	train-logloss:0.164855	valid-logloss:0.187687
[43070]	train-logloss:0.164851	valid-logloss:0.187685
[43080]	train-logloss:0.164846	valid-logloss:0.187683
[43090]	train-logloss:0.164842	valid-logloss:0.187682
[43100]	train-logloss:0.164837	valid-logloss:0.18768
[43110]	train-logloss:0.164833	valid-logloss:0.187679
[43120]	train-logloss:0.164828	valid-logloss:0.187677
[43130]	train-logloss:0.164823	valid-logloss:0.187676
[43140]	train-logloss:0.164818	valid-logloss:0.187675
[43150]	train-logloss:0.164814	valid-logloss:0.187676
[43160]	train-logloss:0.164809	valid-logloss:0.187675
[43170]	train-logloss:0.164804	valid-logloss:0.187675
[43180]	train-logloss:0.164801	valid-logloss:0.187673
[43190]	train-logloss:0.16479

[44540]	train-logloss:0.164192	valid-logloss:0.187506
[44550]	train-logloss:0.164187	valid-logloss:0.187504
[44560]	train-logloss:0.164184	valid-logloss:0.187503
[44570]	train-logloss:0.16418	valid-logloss:0.187502
[44580]	train-logloss:0.164175	valid-logloss:0.187501
[44590]	train-logloss:0.16417	valid-logloss:0.1875
[44600]	train-logloss:0.164165	valid-logloss:0.187499
[44610]	train-logloss:0.164161	valid-logloss:0.187496
[44620]	train-logloss:0.164156	valid-logloss:0.187494
[44630]	train-logloss:0.164151	valid-logloss:0.187492
[44640]	train-logloss:0.164147	valid-logloss:0.187492
[44650]	train-logloss:0.164142	valid-logloss:0.187492
[44660]	train-logloss:0.164137	valid-logloss:0.18749
[44670]	train-logloss:0.164134	valid-logloss:0.187489
[44680]	train-logloss:0.16413	valid-logloss:0.187488
[44690]	train-logloss:0.164126	valid-logloss:0.187487
[44700]	train-logloss:0.164121	valid-logloss:0.187487
[44710]	train-logloss:0.164116	valid-logloss:0.187486
[44720]	train-logloss:0.164112	val

[46070]	train-logloss:0.163521	valid-logloss:0.187328
[46080]	train-logloss:0.163516	valid-logloss:0.187326
[46090]	train-logloss:0.163513	valid-logloss:0.187324
[46100]	train-logloss:0.163508	valid-logloss:0.187323
[46110]	train-logloss:0.163503	valid-logloss:0.187321
[46120]	train-logloss:0.1635	valid-logloss:0.18732
[46130]	train-logloss:0.163495	valid-logloss:0.18732
[46140]	train-logloss:0.163491	valid-logloss:0.18732
[46150]	train-logloss:0.163487	valid-logloss:0.187319
[46160]	train-logloss:0.163483	valid-logloss:0.187316
[46170]	train-logloss:0.163479	valid-logloss:0.187314
[46180]	train-logloss:0.163474	valid-logloss:0.187312
[46190]	train-logloss:0.163471	valid-logloss:0.187311
[46200]	train-logloss:0.163467	valid-logloss:0.187311
[46210]	train-logloss:0.163462	valid-logloss:0.18731
[46220]	train-logloss:0.163458	valid-logloss:0.187311
[46230]	train-logloss:0.163454	valid-logloss:0.187308
[46240]	train-logloss:0.16345	valid-logloss:0.187308
[46250]	train-logloss:0.163445	vali

[47600]	train-logloss:0.162866	valid-logloss:0.187163
[47610]	train-logloss:0.162861	valid-logloss:0.187161
[47620]	train-logloss:0.162857	valid-logloss:0.18716
[47630]	train-logloss:0.162853	valid-logloss:0.187158
[47640]	train-logloss:0.162849	valid-logloss:0.187157
[47650]	train-logloss:0.162844	valid-logloss:0.187156
[47660]	train-logloss:0.16284	valid-logloss:0.187156
[47670]	train-logloss:0.162836	valid-logloss:0.187156
[47680]	train-logloss:0.162831	valid-logloss:0.187153
[47690]	train-logloss:0.162827	valid-logloss:0.187152
[47700]	train-logloss:0.162822	valid-logloss:0.187152
[47710]	train-logloss:0.162817	valid-logloss:0.187152
[47720]	train-logloss:0.162813	valid-logloss:0.187151
[47730]	train-logloss:0.162809	valid-logloss:0.187149
[47740]	train-logloss:0.162805	valid-logloss:0.187148
[47750]	train-logloss:0.162801	valid-logloss:0.187147
[47760]	train-logloss:0.162796	valid-logloss:0.187145
[47770]	train-logloss:0.162791	valid-logloss:0.187143
[47780]	train-logloss:0.162788

[49130]	train-logloss:0.162223	valid-logloss:0.186989
[49140]	train-logloss:0.162218	valid-logloss:0.186987
[49150]	train-logloss:0.162215	valid-logloss:0.186987
[49160]	train-logloss:0.162211	valid-logloss:0.186986
[49170]	train-logloss:0.162206	valid-logloss:0.186985
[49180]	train-logloss:0.162201	valid-logloss:0.186984
[49190]	train-logloss:0.162197	valid-logloss:0.186983
[49200]	train-logloss:0.162193	valid-logloss:0.186982
[49210]	train-logloss:0.162188	valid-logloss:0.18698
[49220]	train-logloss:0.162185	valid-logloss:0.186979
[49230]	train-logloss:0.162181	valid-logloss:0.186978
[49240]	train-logloss:0.162178	valid-logloss:0.186979
[49250]	train-logloss:0.162174	valid-logloss:0.186978
[49260]	train-logloss:0.162169	valid-logloss:0.186977
[49270]	train-logloss:0.162164	valid-logloss:0.186976
[49280]	train-logloss:0.16216	valid-logloss:0.186976
[49290]	train-logloss:0.162156	valid-logloss:0.186975
[49300]	train-logloss:0.162151	valid-logloss:0.186974
[49310]	train-logloss:0.162146

###    

### Ensemble Siamese LSTM + XGBOOST

#### This ensemble got a log loss of 0.16+ on Kaggle 
#### It is a simple averaging ensemble. An average of the predictions from the XGBoost and the Siamese LSTM Network.

In [10]:
#Average ensemble

#read predictions
final_result1 = panda.read_csv('result10.csv')
final_result2 = panda.read_csv('20features_xgb_50k.csv')

#create dataframe and average the predictions
df = panda.DataFrame()
df['test_id'] = final_result1['test_id']
df['is_duplicate'] = (final_result1['is_duplicate'] + final_result2['is_duplicate'])/2


df.to_csv('ensemble_submission.csv', index=False)